In [1]:
import pandas as pd
from pathlib import Path
import os
from matplotlib import pyplot as plt, image as img
import numpy as np
from sklearn.model_selection import train_test_split


from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from keras.layers import BatchNormalization
from keras.layers import (
    Conv2D,
    Conv2DTranspose,
    MaxPool2D,
    ReLU,
    Flatten,
    Dense,
    UpSampling2D,
    Dropout,
    Input, 
    Concatenate,
    GlobalAveragePooling2D, 
    GlobalMaxPooling2D,
)
from skimage.transform import resize
from tensorflow import Tensor
from keras.models import Model, Sequential, load_model
from keras import layers
import tensorflow as tf
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Input
from tensorflow import Tensor
from keras.optimizers import Adam


### Image Read

In [2]:
def load_data():

    p_covid_path = "C:/Users/PC/Documents/FYP/FYP/Code/curated_data/curated_data/2COVID"
    n_covid_path = "C:/Users/PC/Documents/FYP/FYP/Code/curated_data/curated_data/1NonCOVID"
    cap_path = "C:/Users/PC/Documents/FYP/FYP/Code/curated_data/curated_data/3CAP"

    p_covid_images = list(Path(p_covid_path).glob("*.png"))
    n_covid_images = list(Path(n_covid_path).glob("*.png"))
    cap_images = list(Path(cap_path).glob("*.png"))

    return p_covid_images, n_covid_images, cap_images

In [3]:
covid_images, non_covid_images, cap_images = load_data()

print(len(covid_images))
print(len(non_covid_images))
print(len(cap_images))

7593
6893
2618


### Image Resize, Split Training and Validation set

In [4]:
data_dir = "C:/Users/PC/Documents/FYP/FYP/Code/curated_data/curated_data"
img_height, img_width = 180, 180
batch_size = 32

#== Training data ==
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    label_mode="categorical",
    image_size=(img_height,img_width),
    batch_size=batch_size)

Found 17104 files belonging to 3 classes.
Using 13684 files for training.


In [5]:
#== Validation data ==
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    label_mode="categorical",
    image_size=(img_height,img_width),
    batch_size=batch_size)

Found 17104 files belonging to 3 classes.
Using 3420 files for validation.


In [6]:
#classification

class_names=train_ds.class_names
print(class_names)

for i in range(len(class_names)):
    print(class_names[i], " ", i)

['1NonCOVID', '2COVID', '3CAP']
1NonCOVID   0
2COVID   1
3CAP   2


In [7]:
#Defining pre-trained model

resnet_model = Sequential()

pretrained_model = tf.keras.applications.resnet50.ResNet50(include_top=False,
                                                           weights='imagenet',pooling='avg',
                                                           classes=3,
                                                           input_shape=(180,180,3))

for layer in pretrained_model.layers:
    layer.trainable=False

resnet_model.add(pretrained_model)

In [8]:
resnet_model.add(Flatten())
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(128, activation='relu'))
resnet_model.add(BatchNormalization())
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(64, activation='relu'))
resnet_model.add(Dropout(0.5))
resnet_model.add(BatchNormalization())
resnet_model.add(Dense(3, activation='softmax'))

resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout (Dropout)           (None, 512)               0

In [9]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
epochs=20
history = resnet_model.fit(train_ds, validation_data=val_ds, epochs=epochs)